Testing ground for what has become realign.py

In [1]:
from halotools.empirical_models import DimrothWatson
from halotools.utils.rotations3d import rotation_matrices_from_vectors, rotation_matrices_from_angles
from halotools.utils.mcrotations import random_unit_vectors_3d
from halotools.utils.vector_utilities import rotate_vector_collection, normalized_vectors
from halotools.utils import angles_between_list_of_vectors, elementwise_dot
from scipy.spatial.transform import Rotation as R

import numpy as np

# Partial Rotation Matrix

In [2]:
def quat_rotation_matrix_from_vectors(vecA, vecB):
    """
    Compute the quaternion that rotates vecA to vecB.
    """
    vecA = vecA / np.linalg.norm(vecA, axis=1)[:, np.newaxis]
    vecB = vecB / np.linalg.norm(vecB, axis=1)[:, np.newaxis]
    half = (vecA + vecB) / np.linalg.norm(vecA + vecB, axis=1)[:, np.newaxis]
    w = np.sum(vecA * half, axis=1)
    cross_product = np.cross(vecA, half)
    x, y, z = cross_product.T
    quats = np.column_stack((x, y, z, w))
    return R.from_quat(quats)

def slerp_quaternion(quatA, quatB, fraction):
    """
    Perform spherical linear interpolation (Slerp) between two quaternions.
    """
    fraction = np.ones(len(quatA)) * fraction
    dot_product = np.sum(quatA * quatB, axis=1)

    quatB[dot_product < 0] = -quatB[dot_product < 0]
    dot_product = np.abs(dot_product)

    close_mask = dot_product > 0.9995
    far_mask = ~close_mask

    interpolated_quat = np.empty_like(quatA, dtype=np.float64)
    theta_0 = np.arccos(dot_product)
    sin_theta_0 = np.sin(theta_0)

    theta = theta_0 * fraction
    sin_theta = np.sin(theta)

    s0 = np.cos(theta) - dot_product * sin_theta / sin_theta_0
    s1 = sin_theta / sin_theta_0

    interpolated_quat[close_mask] = quatA[close_mask] + fraction[close_mask][:, np.newaxis] * (quatB[close_mask] - quatA[close_mask])
    interpolated_quat[far_mask] = (s0[far_mask][:, np.newaxis] * quatA[far_mask]) + (s1[far_mask][:, np.newaxis] * quatB[far_mask])

    interpolated_quat /= np.linalg.norm(interpolated_quat, axis=1)[:, np.newaxis]
    return interpolated_quat

def partial_rotation_matrices_from_vectors(vecA, vecB, fraction):
    """
    Compute the rotation matrices for partial rotations from vecA to vecB.
    """
    rot_full = quat_rotation_matrix_from_vectors(vecA, vecB)
    quat_full = rot_full.as_quat()
    quat_id = np.array([[0, 0, 0, 1]] * len(vecA))  # Identity quaternions

    quat_partial = slerp_quaternion(quat_id, quat_full, fraction)

    rot_partial = R.from_quat(quat_partial)
    return rot_partial.as_matrix()

def perpendicular_on_plane(vecA, vecB):
    """
    Find the vector perpendicular to vecA on the plane defined by vecA and vecB.
    """
    signs = np.sign(elementwise_dot(vecA, vecB))
    vecC = signs[:,np.newaxis] * vecB
    directions = np.cross(vecA, vecC)
    mats = rotation_matrices_from_angles(np.pi/2 * np.ones(len(vecA)), directions)
    return rotate_vector_collection(mats, vecA)

def align_vectors(vecA, vecB, fraction):
    """
    Rotates vecA some portion of the way towards vecB.
    Negative fraction will align towards the perpendicular vector to vecB.
    Positive fraction will align towards the nearest end of vecB (i.e. vecB or -vecB).
    
    inputs:
    -------
    vecA : array_like, Nx3
        The vectors to be rotated.
    vecB : array_like, Nx3
        The reference direction
    fraction : float or array_like (Nx3)
        The fraction of the way towards vecB to rotate vecA.
        If float, the same fraction will be used for all vectors.
        If array_like, the fraction for each vector.
    """
    fraction = np.ones(len(vecA)) * fraction
    ref_vecs = np.array(vecB)

    # Compute the rotation matrices
    perpendicular_mask = fraction < 0
    if perpendicular_mask.any():
        # Not that vecB is the first vector passed in
        # This is because we want the vector perpendicular to vecB on the plane defined by vecB and vecA
        ref_vecs[perpendicular_mask] = perpendicular_on_plane(vecB[perpendicular_mask], vecA[perpendicular_mask])

    # Ensure we use just the nearest side of the reference vector
    signs = np.sign(elementwise_dot(vecA, ref_vecs))
    ref_vecs = signs[:,np.newaxis] * ref_vecs
    mats = partial_rotation_matrices_from_vectors(vecA, ref_vecs, abs(fraction))
    
    return rotate_vector_collection(mats, vecA)

# Perform The Rotations

## Align Towards Perfect Alignment

In [3]:
vecA, vecB = random_unit_vectors_3d(2)
vecB *= np.sign(elementwise_dot(vecA, vecB))
mat = rotation_matrices_from_vectors([vecA], [vecB])
half_mat = partial_rotation_matrices_from_vectors([vecA], [vecB], 0.5)
quarter_mat = partial_rotation_matrices_from_vectors([vecA], [vecB], 0.25)

print("Rotate A into B")
print("A:\t\t", vecA)
print("B:\t\t", vecB)
print("Rotated A:\t", rotate_vector_collection(mat, [vecA])[0])

print("\nHalf Rotations")
print("A:\t\t\t", vecA)
print("B:\t\t\t", vecB)
vecC = normalized_vectors( rotate_vector_collection(half_mat, [vecA]) )[0]
print("First Rotation of A:\t", vecC)
vecC = normalized_vectors( rotate_vector_collection(half_mat, [vecC]) )[0]
print("Second Rotation of A:\t", vecC)
print("Angle between B and Second Rotation of A:", angles_between_list_of_vectors([vecB], [vecC])[0])

print("\nQuarter Rotations")
print("A:\t\t\t", vecA)
print("B:\t\t\t", vecB)
vecC = normalized_vectors( rotate_vector_collection(quarter_mat, [vecA]) )[0]
print("First Rotation of A:\t", vecC)
vecC = normalized_vectors( rotate_vector_collection(quarter_mat, [vecC]) )[0]
print("Second Rotation of A:\t", vecC)
vecC = normalized_vectors( rotate_vector_collection(quarter_mat, [vecC]) )[0]
print("Third Rotation of A:\t", vecC)
vecC = normalized_vectors( rotate_vector_collection(quarter_mat, [vecC]) )[0]
print("Fourth Rotation of A:\t", vecC)
print("Angle between B and Fourth Rotation of A:", angles_between_list_of_vectors([vecB], [vecC])[0])

Rotate A into B
A:		 [-0.16635786  0.89698435 -0.40956579]
B:		 [0.21079795 0.73857608 0.64036676]
Rotated A:	 [0.21079795 0.73857608 0.64036676]

Half Rotations
A:			 [-0.16635786  0.89698435 -0.40956579]
B:			 [0.21079795 0.73857608 0.64036676]
First Rotation of A:	 [0.02689488 0.98983147 0.13967938]
Second Rotation of A:	 [0.21079795 0.73857608 0.64036676]
Angle between B and Second Rotation of A: 0.0

Quarter Rotations
A:			 [-0.16635786  0.89698435 -0.40956579]
B:			 [0.21079795 0.73857608 0.64036676]
First Rotation of A:	 [-0.07297465  0.98728507 -0.14121931]
Second Rotation of A:	 [0.02689488 0.98983147 0.13967938]
Third Rotation of A:	 [0.12437387 0.90439721 0.40816274]
Fourth Rotation of A:	 [0.21079795 0.73857608 0.64036676]
Angle between B and Fourth Rotation of A: 0.0


In [4]:
# Double check with large lists of vectors
N = 1000
vecA_large, vecB_large = random_unit_vectors_3d(N), random_unit_vectors_3d(N)
vecB_large *= np.sign(elementwise_dot(vecA_large, vecB_large))[:, np.newaxis]
mat_large = rotation_matrices_from_vectors(vecA_large, vecB_large)
quarter_mat_large = partial_rotation_matrices_from_vectors(vecA_large, vecB_large, 0.25)

# First, check that the large list of vectors is rotated correctly using the normal rotation matrix
vecC_large = rotate_vector_collection(mat_large, vecA_large)
dots = elementwise_dot(vecC_large, vecB_large)
print("Dot Is One:\t\t\t", np.allclose(dots, 1, rtol=1e-5))

# Now, check that the large list of vectors is rotated correctly using the partial rotation matrix
print("\nQuarter Rotations")
vecC_large = rotate_vector_collection(quarter_mat_large, vecA_large)
dots = elementwise_dot(vecC_large, vecB_large)
print("First Rotation Dot Is Not One:\t", not np.allclose(dots, 1, rtol=1e-5))
vecC_large = rotate_vector_collection(quarter_mat_large, vecC_large)
dots = elementwise_dot(vecC_large, vecB_large)
print("Second Rotation Dot Is Not One:\t", not np.allclose(dots, 1, rtol=1e-5))
vecC_large = rotate_vector_collection(quarter_mat_large, vecC_large)
dots = elementwise_dot(vecC_large, vecB_large)
print("Third Rotation Dot Is Not One:\t", not np.allclose(dots, 1, rtol=1e-5))
vecC_large = rotate_vector_collection(quarter_mat_large, vecC_large)
dots = elementwise_dot(vecC_large, vecB_large)
print("Fourth Rotation Dot Is One:\t", np.allclose(dots, 1, rtol=1e-5))

Dot Is One:			 True

Quarter Rotations
First Rotation Dot Is Not One:	 True
Second Rotation Dot Is Not One:	 True
Third Rotation Dot Is Not One:	 True
Fourth Rotation Dot Is One:	 True


## Align Towards Perpendicular

In [5]:
A = [vecA, vecA]
B = [vecB, vecB]
perp_vecB = perpendicular_on_plane(B, A)

print("Vecs")
print("A:\t", A)
print("B:\t", B)
print("B_perp:\t", perp_vecB)

print("Angles")
ang1 = np.arccos( elementwise_dot(A, B) )
ang2 = np.arccos( elementwise_dot(A, perp_vecB) )
ang3 = np.arccos( elementwise_dot(B, perp_vecB) )
print("Angle A-B:\t", ang1)
print("Angle A-B_perp:\t", ang2)
print("Angle B-B_perp:\t", ang3)

Vecs
A:	 [array([-0.16635786,  0.89698435, -0.40956579]), array([-0.16635786,  0.89698435, -0.40956579])]
B:	 [array([0.21079795, 0.73857608, 0.64036676]), array([0.21079795, 0.73857608, 0.64036676])]
B_perp:	 [[-0.26137964  0.67382105 -0.69111929]
 [-0.26137964  0.67382105 -0.69111929]]
Angles
Angle A-B:	 [1.19700135 1.19700135]
Angle A-B_perp:	 [0.37379497 0.37379497]
Angle B-B_perp:	 [1.57079633 1.57079633]


In [6]:
holdA = rotate_vector_collection(half_mat, [vecA])[0]
holdB = rotate_vector_collection(np.transpose(half_mat, axes=(0,2,1)), [vecA])[0]

print(holdA)
print(holdB)
print(angles_between_list_of_vectors([holdA], [holdB])[0])
print(angles_between_list_of_vectors([vecA], [holdA])[0])
print(angles_between_list_of_vectors([vecA], [holdB])[0])

[0.02689488 0.98983147 0.13967938]
[-0.30177837  0.49231187 -0.81643054]
1.1970013533203097
0.5985006766601543
0.5985006766601546


In [7]:
# Check with large lists of vectors
# use vecA_large and vecB_large from before
vecB_large_perp = perpendicular_on_plane(vecB_large, vecA_large)
mat_large = rotation_matrices_from_vectors(vecA_large, vecB_large_perp)
quarter_mat_large = partial_rotation_matrices_from_vectors(vecA_large, vecB_large_perp, 0.25)

# First, check that the large list of vectors is rotated correctly using the normal rotation matrix
vecD_large = rotate_vector_collection(mat_large, vecA_large)
dots = elementwise_dot(vecD_large, vecB_large_perp)
print("Dot Is One:\t\t\t", np.allclose(dots, 1, rtol=1e-5))

# Now, check that the large list of vectors is rotated correctly using the partial rotation matrix
print("\nQuarter Rotations")
vecC_large = rotate_vector_collection(quarter_mat_large, vecA_large)
dots = elementwise_dot(vecD_large, vecB_large)
print("First Rotation Dot Is Not One:\t", not np.allclose(dots, 1, rtol=1e-5))
vecC_large = rotate_vector_collection(quarter_mat_large, vecD_large)
dots = elementwise_dot(vecD_large, vecB_large)
print("Second Rotation Dot Is Not One:\t", not np.allclose(dots, 1, rtol=1e-5))
vecC_large = rotate_vector_collection(quarter_mat_large, vecD_large)
dots = elementwise_dot(vecD_large, vecB_large)
print("Third Rotation Dot Is Not One:\t", not np.allclose(dots, 1, rtol=1e-5))
vecC_large = rotate_vector_collection(quarter_mat_large, vecD_large)
dots = elementwise_dot(vecD_large, vecB_large_perp)
print("Fourth Rotation Dot Is One:\t", np.allclose(dots, 1, rtol=1e-5))

Dot Is One:			 True

Quarter Rotations
First Rotation Dot Is Not One:	 True
Second Rotation Dot Is Not One:	 True
Third Rotation Dot Is Not One:	 True
Fourth Rotation Dot Is One:	 True


# Tests - Rotate Mixed

## Single Float Fraction
This should work the same as what we've seen

In [8]:
m = 1_000_000
ref_x = np.array([ np.ones(m), np.zeros(m), np.zeros(m) ]).T
ref_y = np.array([ np.zeros(m), np.ones(m), np.zeros(m) ]).T
ref_z = np.array([ np.zeros(m), np.zeros(m), np.ones(m) ]).T
vecs = normalized_vectors( np.array([ np.random.uniform(0, 1, size=m), np.random.uniform(0, 1, size=m), np.zeros(m) ]).T )
initial_angles = angles_between_list_of_vectors(vecs, ref_x)
complement_angles = np.pi/2 - initial_angles

### Towards Reference

In [9]:
# Rotate towards the reference vectors
rot_vecs = align_vectors(vecs, ref_x, 1)                        # Perfect Rotation
assert np.allclose(rot_vecs, ref_x, rtol=1e-5)
rot_vecs = align_vectors(vecs, ref_x, 0.5)                      # Half Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_x)
assert np.allclose(rot_angles, initial_angles / 2, rtol=1e-5)
rot_vecs = align_vectors(vecs, ref_x, 0)                        # No Rotation
assert np.allclose(rot_vecs, vecs, rtol=1e-5)
rand_fracs = np.random.rand(m)
rot_vecs = align_vectors(vecs, ref_x, rand_fracs)         # Random Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_x)
assert np.allclose(rot_angles, initial_angles * (1-rand_fracs), rtol=1e-3)

### Towards Perpendicular

In [10]:
# Rotate towards the vectors perpendicular to reference
rot_vecs = align_vectors(vecs, ref_x, -1)                        # Perfect Rotation
assert np.allclose(rot_vecs, ref_y, rtol=1e-5)
rot_vecs = align_vectors(vecs, ref_x, -0.5)                      # Half Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_x)
assert np.allclose(rot_angles, initial_angles + (complement_angles / 2 ), rtol=1e-5)
rot_vecs = align_vectors(vecs, ref_x, 0)                        # No Rotation
assert np.allclose(rot_vecs, vecs, rtol=1e-5)
rand_fracs = np.random.rand(m)
rot_vecs = align_vectors(vecs, ref_x, -rand_fracs)         # Random Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_x)
assert np.allclose(rot_angles, initial_angles + ( complement_angles * rand_fracs ), rtol=1e-3)

## Array of Fractions

### Align Towards

In [11]:
# Rotate towards parallel
# Pass in random array of fractions
rand_fracs = np.random.uniform(0, 1, size=m)
ref_vecs = normalized_vectors( np.array( [ np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m) ] ).T )
vecs = normalized_vectors( np.array( [ np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m) ] ).T )
close_vecs = np.sign(elementwise_dot(vecs, ref_vecs))[:, np.newaxis] * vecs                     # For looking at the smaller angle
initial_angles = angles_between_list_of_vectors(close_vecs, ref_vecs)
complement_angles = np.pi/2 - initial_angles
perp_mask = rand_fracs < 0

In [12]:
rot_vecs = align_vectors(vecs, ref_vecs, rand_fracs)                                            # Random Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_vecs)                                 # Check the angles
expected_angles = initial_angles * (1 - np.abs(rand_fracs))                                     # Reduce angle for those rotating towards parallel
expected_angles = np.minimum(expected_angles, np.pi-expected_angles)                            # Ensure angles are less than 90 degrees
rot_angles = np.minimum(rot_angles, np.pi-rot_angles)                                           # Ensure angles are less than 90 degrees
assert np.allclose(rot_angles, expected_angles, rtol=1e-3)
assert(rot_angles <= np.pi/2).all()
assert (rot_angles <= initial_angles).all()
assert (rot_angles >= 0).all()

### Align Perpendicular

In [13]:
# Rotate towards perpendicular
# Pass in random array of fractions
rand_fracs = np.random.uniform(-1, 0, size=m)
ref_vecs = normalized_vectors( np.array( [ np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m) ] ).T )
vecs = normalized_vectors( np.array( [ np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m) ] ).T )
close_vecs = np.sign(elementwise_dot(vecs, ref_vecs))[:, np.newaxis] * vecs                     # For looking at the smaller angle
initial_angles = angles_between_list_of_vectors(close_vecs, ref_vecs)
complement_angles = np.pi/2 - initial_angles
perp_mask = rand_fracs < 0

In [14]:
rot_vecs = align_vectors(vecs, ref_vecs, rand_fracs)                                            # Random Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_vecs)                                 # Check the angles
delta_theta = complement_angles * np.abs(rand_fracs)                                            # Increase angle for those rotating towards perpendicular
expected_angles = initial_angles + delta_theta                                                  # Reduce angle for those rotating towards parallel
expected_angles = np.minimum(expected_angles, np.pi-expected_angles)                            # Ensure angles are less than 90 degrees
rot_angles = np.minimum(rot_angles, np.pi-rot_angles)                                           # Ensure angles are less than 90 degrees
assert np.allclose(rot_angles, expected_angles, rtol=1e-3)
assert(rot_angles >= initial_angles).all()
assert (rot_angles <= np.pi/2).all()
assert (rot_angles >= 0).all()

### Mixed Array

In [20]:
# Rotate either towards parallel or towards perpendicular
# Pass in random array of fractions
rand_fracs = np.random.uniform(-1, 1, size=m)
ref_vecs = normalized_vectors( np.array( [ np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m) ] ).T )
vecs = normalized_vectors( np.array( [ np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m), np.random.uniform(-1, 1, size=m) ] ).T )
close_vecs = np.sign(elementwise_dot(vecs, ref_vecs))[:, np.newaxis] * vecs                     # For looking at the smaller angle
initial_angles = angles_between_list_of_vectors(close_vecs, ref_vecs)
complement_angles = np.pi/2 - initial_angles
perp_mask = rand_fracs < 0

In [32]:
rot_vecs = align_vectors(vecs, ref_vecs, rand_fracs)                                            # Random Rotation
rot_angles = angles_between_list_of_vectors(rot_vecs, ref_vecs)                                 # Check the angles
# Handle the parallel cases
expected_angles = initial_angles * (1 - np.abs(rand_fracs))                                     # Reduce angle for those rotating towards parallel
# Handle the perpendicular cases
delta_theta = complement_angles[perp_mask] * np.abs(rand_fracs[perp_mask])                      # Increase angle for those rotating towards perpendicular
expected_angles[perp_mask] = initial_angles[perp_mask] + delta_theta                            # Reduce angle for those rotating towards parallel

expected_angles = np.minimum(expected_angles, np.pi-expected_angles)                            # Ensure angles are less than 90 degrees
rot_angles = np.minimum(rot_angles, np.pi-rot_angles)                                           # Ensure angles are less than 90 degrees
assert(np.allclose(rot_angles[~perp_mask], expected_angles[~perp_mask], rtol=1e-3))
assert(np.allclose(rot_angles[perp_mask], expected_angles[perp_mask], rtol=1e-3))
assert (rot_angles[~perp_mask] <= initial_angles[~perp_mask]).all()                              # Make sure all + fractions rotate towards
assert (rot_angles[perp_mask] >= initial_angles[perp_mask]).all()                               # Make sure all - fractions rotate towards
assert (rot_angles <= np.pi/2).all()
assert (rot_angles >= 0).all()